# Circulant Matrices

In this lecture, I want to introduce you to a new type of matrix: **circulant** matrices.  Like Hermitian matrices, they have orthonormal eigenvectors, but unlike Hermitian matrices we know *exactly* what their eigenvectors are!  Moreover, their eigenvectors are closely related to the famous Fourier transform and Fourier series.  Even more importantly, it turns out that circulant matrices and the eigenvectors lend themselves to **incredibly efficient** algorithms called FFTs, that play a central role in much of computational science and engineering.

![a ring of springs](https://github.com/stevengj/1806-spring17/raw/master/lectures/cyclic-springs.png) 

Consider a system of $n$ identical masses $m$ connected by springs $k$, sliding around a *circle* without friction.   Similar to lecture 26, the vector $\vec{s}$ of displacements satifies $m\frac{d^2\vec{s}}{dt^2} = -kA\vec{s}$, where $A$ is the $n \times n$ matrix:

$$
A = \begin{pmatrix} 2 & -1 & & & & & -1 \\
                   -1 & 2 &-1& & & & \\
                      &-1 &2&-1& & & \\
                      &   &\ddots&\ddots&\ddots& & \\ 
                   & & &-1 & 2  &-1 & \\
                   & & & & -1 &2 & -1 \\
                   -1 &   &  &  & &-1 &2
    \end{pmatrix}
$$

(This matrix is real-symmetric and, less obviously, positive semidefinite.  So, it should have orthogonal eigenvectors and real eigenvalues $\lambda \ge 0$.)

For example, if $n = 7$:

In [8]:
A = [ 2 -1  0  0  0  0 -1
     -1  2 -1  0  0  0  0
      0 -1  2 -1  0  0  0
      0  0 -1  2 -1  0  0
      0  0  0 -1  2 -1  0
      0  0  0  0 -1  2 -1
     -1  0  0  0  0 -1  2]

A =

   2  -1   0   0   0   0  -1
  -1   2  -1   0   0   0   0
   0  -1   2  -1   0   0   0
   0   0  -1   2  -1   0   0
   0   0   0  -1   2  -1   0
   0   0   0   0  -1   2  -1
  -1   0   0   0   0  -1   2



This matrix has a very special pattern: *every row is the same as the previous row, just shifted to the right by 1* (wrapping around "cyclically" at the edges).  That is, each row is a [circular shift](https://en.wikipedia.org/wiki/Circular_shift) of the first row.

This is called a [circulant matrix](https://en.wikipedia.org/wiki/Circulant_matrix).  A $4\times 4$ circulant matrix looks like:

$$
C = \begin{pmatrix}
c_0 & c_1 & c_2 & c_3 \\
c_3 & c_0 & c_1 & c_2 \\
c_2 & c_3 & c_0 & c_1 \\
c_1 & c_2 & c_3 & c_0
\end{pmatrix}
$$

The general form of an $n \times n$ circulant matrix $C$ is:

$$
C = \begin{pmatrix}
c_0 & c_1 & c_2 & \cdots & c_{n-1} \\
c_{n-1} & c_0 & c_1 & c_2 & \cdots \\
c_{n-2} & c_{n-1} & c_0 & \cdots \\
\ddots & \ddots & \ddots & \ddots & \ddots \\
c_1 & c_2 & \cdots & c_{n-1} & c_0 
\end{pmatrix}
$$

When working with circulant matrix, it is convenient to number entries from $0$ to $n-1$ rather than from $1$ to $n$!

## Multiplying by circulant matrices: Convolutions

Suppose we have an $n \times n$ circulant matrix $C$ that we want to multiply by a vector $x = (x_0, x_1, \ldots, x_n)$.   It turns out that the result is a very special kind of operation:

$$
y = Cx = \begin{pmatrix}
c_0 & c_1 & c_2 & \cdots & c_{n-1} \\
c_{n-1} & c_0 & c_1 & c_2 & \cdots \\
c_{n-2} & c_{n-1} & c_0 & \cdots \\
\ddots & \ddots & \ddots & \ddots & \ddots \\
c_1 & c_2 & \cdots & c_{n-1} & c_0 
\end{pmatrix} \begin{pmatrix} x_0 \\ x_1 \\ \vdots \\ x_{n-1} \end{pmatrix}
$$

Let's write down a formula for the entries of $y$:

$$
y_0 = c_0 x_0 + c_1 x_1 + c_2 x_2 + \cdots \\
y_1 = c_{n-1} x_0 + c_0 x_1 + c_1 x_2 + \cdots \\
y_2 = c_{n-2} x_0 + c_{n-1} x_1 + c_0 x_2 + \cdots
$$

Can you see the pattern?  This is one of those cases that is actually clearer if we write out the summation:

$$
y_k = \sum_{j=0}^{n-1} c_{j-k} x_j
$$

There is a slight problem with this formula: the subscript $j-k$ can be $< 0$!  No problem: we just *interpret the subscript periodically*, i.e. we let $c_{-1} = c_{n-1}$, $c_{-2} = c_{n-2}$, and so on.  Equivalently, we define $c_{j\pm n} = c_j$.   (We could say that the subscripts are [modulo n](https://en.wikipedia.org/wiki/Modular_arithmetic).)

Multiplying by a circulant matrix is equivalent to a very famous operation called a [circular convolution](https://en.wikipedia.org/wiki/Circular_convolution).  Convolution operations, and hence circulant matrices, show up in lots of applications: **digital signal processing**, **image compression**, **physics/engineering simulations**, **number theory** and **cryptography**, and so on.

# Eigenvectors of circulant matrices

One amazing property of circulant matrices is that **the eigenvectors are always the same**.  The eigen-*values* are different for each C, but since we know the eigenvectors they are easy to diagonalize.

We can actually see one eigenvector right away.  Let's call it $x^{(0)}$:

$$
x^{(0)} = \begin{pmatrix} 1 \\ 1 \\ 1 \\ \vdots \\ 1 \end{pmatrix}
$$

This is an eigenvector because multiplying $C x^{(0)}$ **simply sums each row of C**.  But since each row of C contains the same entries (just in a different order), the sum is the same:

$$
C x^{(0)} = \underbrace{(c_0 + c_1 + \cdots + c_{n-1})}_{\lambda_0} x^{(0)}
$$

Thus, one of the eigenvalues $\lambda_0$ of $C$ is simply the sum of the row entries.

For our example matrix $A$ above, this sum is $-1 + 2 + -1 = 0$, so $A$ is a *singular* matrix with an eigenvalue zero.

In [9]:
eig(A)

ans =

  -9.4114e-16
   7.5302e-01
   7.5302e-01
   2.4450e+00
   2.4450e+00
   3.8019e+00
   3.8019e+00



In [10]:
function C = Circulant1(a)
    % C = Circulant1(a) is a circulant matrix with first row equal to a.
    n = length(a);
    C = zeros(n,n);
    for i=1:n
        for j=1:n
        C(i,j) = a(rem(n-i+j,n)+1);
        end
    end
end
function C = Circulant2(a)
    % C = Circulant2(a) is a circulant matrix with first row equal to a.
    n = length(a);
    C = zeros(n,n);
    C(1,:) = a;
    for i=2:n
        C(i,:) = [ C(i-1,n) C(i-1,1:n-1) ];
    end
end

In [11]:
A=[1 2 3 ;3 4 5;2 3 4];

i = sqrt(-1)
F = [1 1 1 1; 1 -i -1 i; 1 -1 1 -1; 1 i -1 -i]

i =  0 + 1i
F =

   1 + 0i   1 + 0i   1 + 0i   1 + 0i
   1 + 0i  -0 - 1i  -1 + 0i   0 + 1i
   1 + 0i  -1 + 0i   1 + 0i  -1 + 0i
   1 + 0i   0 + 1i  -1 + 0i  -0 - 1i



In [12]:
function F=f(n)
    F = ones(n,n);
    F(:,2) = exp(-2*pi*sqrt(-1)/n).^(0:n-1)'';
    for k=3:n
        F(:,k) = F(:,2).*F(:,k-1);
    end
end

In [18]:
format rat
F=f(4);
F

F =

          1 +         0i          1 -         0i          1 -         0i          1 -         0i
          1 +         0i          0 -         1i         -1 -         0i          0 +         1i
          1 +         0i         -1 -         0i          1 +         0i         -1 -         0i
          1 +         0i          0 +         1i         -1 -         0i          0 -         1i



In [2]:
function y = DFT(x)
    % y = DFT(x)
    % y is the discrete Fourier transform of a column n-vector x.
    n = length(x);
    y = x(1)*ones(n,1);
    if n > 1
    v = exp(-2*pi*sqrt(-1)/n).^(0:n-1)';
    for k=2:n
        z = rem((k-1)*(0:n-1)’,n )+1;
        y = y + v(z)*x(k);
    end
end

parse error:

  syntax error

>>>         z = rem((k-1)*(0:n-1)’,n )+1;
                                 ^

error: 'y' undefined near line 1 column 13
parse error:

  syntax error

>>>     end
          ^

parse error:

  syntax error

>>> end
      ^

